In [1]:
from trainer import sequence_accuracy
from data import Data
import pandas as pd
import numpy as np
import os
import torch
from config import TransformerTestConfig
from tokenizer import Tokenizer
from fn_utils import get_model

/global/homes/r/ritesh11/.local/perlmutter/pytorch2.3.1/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/global/homes/r/ritesh11/.local/perlmutter/pytorch2.3.1/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
# Special tokens & coressponding ids
BOS_IDX, PAD_IDX, EOS_IDX, UNK_IDX, SEP_IDX = 0, 1, 2, 3, 4
special_symbols = ['<S>', '<PAD>', '</S>', '<UNK>', '<SEP>']

In [3]:
config = TransformerTestConfig(
    model_name="transformer_64x2",
    root_dir="/pscratch/sd/r/ritesh11/SYMBA_test/Vanilla",
    data_dir="/pscratch/sd/r/ritesh11/SYMBA_test/SYMBA_test",
    device='cuda',
    embedding_size=512,
    nhead=8,
    num_encoder_layers=3,
    num_decoder_layers=3,
    dropout=0.1,
    hidden_dim=8192,
    src_max_len=280,
    tgt_max_len=323,
    src_voc_size=None,
    tgt_voc_size=None,
    seed=42,
    to_replace=False,
    index_pool_size=200,
    momentum_pool_size=200,
    is_prefix=False
)

In [4]:
df_train = pd.read_csv(config.data_dir+"train.csv")
df_test = pd.read_csv(config.data_dir+"test.csv")
df_valid = pd.read_csv(config.data_dir+"valid.csv")

df = pd.concat([df_train,df_valid,df_test]).reset_index(drop=True)

In [5]:
tokenizer = Tokenizer(df, config.index_pool_size,config.momentum_pool_size, special_symbols, UNK_IDX,config.to_replace)
src_vocab = tokenizer.build_src_vocab(config.seed)
src_itos = {value: key for key, value in src_vocab.get_stoi().items()}
tgt_vocab = tokenizer.build_tgt_vocab()
tgt_itos = {value: key for key, value in tgt_vocab.get_stoi().items()}

Processing target vocab: 100%|██████████| 15552/15552 [00:01<00:00, 14230.70it/s]


In [6]:
config.src_voc_size = len(src_vocab)
config.tgt_voc_size = len(tgt_vocab)

In [7]:
model = get_model(config)

/global/homes/r/ritesh11/.local/perlmutter/pytorch2.3.1/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
datasets = Data.get_data(
            df_train, df_test, df_valid, config, tokenizer,src_vocab, tgt_vocab)

test_ds = datasets['test']

In [9]:
preds = sequence_accuracy(config,test_ds,tgt_itos,test_size=len(test_ds))

Using epoch 20 model for predictions.


Seq_Acc_Cal:   3%|▎         | 49/1555 [00:22<11:25,  2.20it/s, seq_accuracy=0.898]


KeyboardInterrupt: 